# 3.1 输入输出重定向
输入重定向是指把文件导入到命令中，而输出重定向则是指把原本要输出到屏幕的数据信息写入到指定文件中。输出重定向分为了标准输出重定向和错误输出重定向两种不同的技术，以及清空写入与追加写入两种模式。

>标准输入重定向（STDIN，文件描述符为0）：默认从键盘输入，也可从其他文件或命令中输入。
>标准输出重定向（STDOUT，文件描述符为1）：默认输出到屏幕。
>错误输出重定向（STDERR，文件描述符为2）：默认输出到屏幕。

 输入重定向中用到的符号及其作用

符号|作用
----|----
命令 < 文件|将文件作为命令的标准输入
命令 << 分界符|从标准输入中读入，直到遇见分界符才停止
命令 < 文件1 > 文件2|将文件1作为命令的标准输入并将标准输出到文件2

 输出重定向中用到的符号及其作用

符号|作用
----|----
命令 > 文件|将标准输出重定向到一个文件中（清空原有文件的数据）
命令 2> 文件|将错误输出重定向到一个文件中（清空原有文件的数据）
命令 >> 文件|将标准输出重定向到一个文件中（追加到原有内容的后面）
命令 2>> 文件|将错误输出重定向到一个文件中（追加到原有内容的后面）
命令 >> 文件 2>&1 或 命令 &>> 文件|将标准输出与错误输出共同写入到文件中（追加到原有内容的后面）

通过标准输出重定向将man bash命令原本要输出到屏幕的信息写入到文件readme.txt中，然后显示readme.txt文件中的内容。具体命令如下：
```bash
[root@linuxprobe ~]# man bash > readme.txt
[root@linuxprobe ~]# cat readme.txt 
BASH(1)                     General Commands Manual                    BASH(1)
NAME
bash - GNU Bourne-Again Shell

SYNOPSIS
bash [options] [file]

COPYRIGHT
Bash is Copyright (C) 1989-2011 by the Free Software Foundation, Inc.

DESCRIPTION
Bash  is  an  sh-compatible  command language interpreter that executes
commands read from the standard input or from a file.  Bash also incor‐
porates useful features from the Korn and C shells (ksh and csh).

Bash  is  intended  to  be a conformant implementation of the Shell and
Utilities portion  of  the  IEEE  POSIX  specification  (IEEE  Standard
1003.1).  Bash can be configured to be POSIX-conformant by default.

………………省略部分输出信息………………
```

# 3.2 管道命令符
管道命令符的作用也可以用一句话来概括“把前一个命令原本要输出到屏幕的标准正常数据当作是后一个命令的标准输入”。

在修改用户密码时，通常都需要输入两次密码以进行确认，这在编写自动化脚本时将成为一个非常致命的缺陷。通过把管道符和passwd命令的--stdin参数相结合，我们可以用一条命令来完成密码重置操作：
```bash
[root@linuxprobe ~]# echo "linuxprobe" | passwd --stdin root
Changing password for user root.
passwd: all authentication tokens updated successfully.
```
比如，在发送电子邮件时，默认采用交互式的方式来进行，我们完全可以利用一条结合了管道符的命令语句，把编辑好的内容与标题一起“打包”，最终用这一条命令实现邮件的发送。
```bash
[root@linuxprobe ~]# echo "Content" | mail -s "Subject" linuxprobe
[root@linuxprobe ~]# su - linuxprobe
Last login: Fri Jul 10 09:44:07 CST 2017 on :0
[linuxprobe@linuxprobe ~]$ mail
Heirloom Mail version 12.5 7/5/10. Type ? for help.
"/var/spool/mail/linuxprobe": 1 message 1 new
>N 1 root Sun Aug 30 17:33 18/578 "Subject"
```

# 3.3 命令行通配符
通配符就是通用的匹配信息的符号，比如星号（\*）代表匹配零个或多个字符，问号（?）代表匹配单个字符，中括号内加上数字[0-9]代表匹配0～9之间的单个数字的字符，而中括号内加上字母[abc]则是代表匹配a、b、c三个字符中的任意一个字符。

匹配所有在/dev目录中且以sda开头的文件：
```bash
[root@linuxprobe ~]# ls -l /dev/sda*
brw-rw----. 1 root disk 8, 0 May 4 15:55 /dev/sda
brw-rw----. 1 root disk 8, 1 May 4 15:55 /dev/sda1
brw-rw----. 1 root disk 8, 2 May 4 15:55 /dev/sda2
```

# 3.4 常用的转义字符
4个最常用的转义字符如下所示。
- 反斜杠（\）：使反斜杠后面的一个变量变为单纯的字符串。
- 单引号（''）：转义其中所有的变量为单纯的字符串。
- 双引号（""）：保留其中的变量属性，不进行转义处理。
- 反引号（\`\`）：把其中的命令执行后返回结果。

# 3.5 重要的环境变量
在Linux系统中，变量名称一般都是大写的，这是一种约定俗成的规范。我们可以直接通过变量名称来提取到对应的变量值。Linux系统中的环境变量是用来定义系统运行环境的一些参数。

命令在Linux中的执行分为4个步骤：
1. 判断用户是否以绝对路径或相对路径的方式输入命令（如/bin/ls），如果是的话则直接执行。
2. Linux系统检查用户输入的命令是否为“别名命令”，即用一个自定义的命令名称来替换原本的命令名称。
    可以用alias命令来创建一个属于自己的命令别名，格式为“alias 别名=命令”。若要取消一个命令别名，则是用unalias命令，格式为“unalias 别名”。
3. Bash解释器判断用户输入的是内部命令还是外部命令。内部命令是解释器内部的指令，会被直接执行；而用户在绝大部分时间输入的是外部命令，这些命令交由步骤4继续处理。
4. 系统在多个路径中查找用户输入的命令文件，而定义这些路径的变量叫作PATH，可以简单地把它理解成是“解释器的小助手”，作用是告诉Bash解释器待执行的命令可能存放的位置，然后Bash解释器就会乖乖地在这些位置中逐个查找。PATH是由多个路径值组成的变量，每个路径值之间用冒号间隔，对这些路径的增加和删除操作将影响到Bash解释器对Linux命令的查找。

Linux系统中最重要的10个环境变量

变量名称|作用
--------|----
HOME|用户的主目录（即家目录）
SHELL|用户在使用的Shell解释器名称
HISTSIZE|输出的历史命令记录条数
HISTFILESIZE|保存的历史命令记录条数
MAIL|邮件保存路径
LANG|系统语言、语系名称
RANDOM|生成一个随机数字
PS1|Bash解释器的提示符
PATH|定义解释器搜索用户执行命令的路径
EDITOR|用户默认的文本编辑器

我们使用下述命令来查看HOME变量在不同用户身份下都有哪些值（su是用于切换用户身份的命令，将在第5章跟大家见面）：
```bash
[root@linuxprobe ~]# echo $HOME
/root
[root@linuxprobe ~]# su - linuxprobe
Last login: Fri Feb 27 19:49:57 CST 2017 on pts/0
[linuxprobe@linuxprobe ~]$ echo $HOME
/home/linuxprobe
```

如果工作需要，可以使用export命令将其提升为全局变量，这样其他用户也就可以使用它了：
```bash
[root@linuxprobe workdir]# su linuxprobe
Last login: Fri Mar 20 20:52:10 CST 2017 on pts/0
[linuxprobe@linuxprobe ~]$ cd $WORKDIR
[linuxprobe@linuxprobe ~]$ echo $WORKDIR
[linuxprobe@linuxprobe ~]$ exit
[root@linuxprobe ~]# export WORKDIR
[root@linuxprobe ~]# su linuxprobe
Last login: Fri Mar 20 21:52:10 CST 2017 on pts/0
[linuxprobe@linuxprobe ~]$ cd $WORKDIR
[linuxprobe@linuxprobe workdir]$ pwd
/home/workdir
```